In [1]:
import tkinter as tk
import requests
from tkinter import filedialog
from functools import partial
from googletrans import Translator
from tkinter import messagebox as msg
from tkinter.ttk import Notebook
from openpyxl import *
from tkinter import *
#from ipynb.fs.full.Total_window import *
import os
#from import_ipynb import *

In [2]:
###############################  Main Windows  ################
#File manupulation main
class Editor(tk.Tk): 
    
    def __init__(self):
        
        super().__init__()
        
       # root = tk.Tk()   
        
        self.FONT_SIZE = 12
        self.AUTOCOMPLETE_WORDS = ["def","import","if","else","while","for","try:","except:"
                                   ,"print(","True","False",]
        self.WINDOW_TITLE = "Text Editor"

        self.open_file = ""

        self.title(self.WINDOW_TITLE)
        self.geometry("800x600")

        self.menubar = tk.Menu(self, bg="lightgrey", fg="black")

        self.file_menu = tk.Menu(self.menubar, tearoff=0, bg="lightgrey", fg="black")
        self.file_menu.add_command(label="New", command=self.file_new, accelerator="Ctrl+N")
        self.file_menu.add_command(label="Open", command=self.file_open, accelerator="Ctrl+O")
        self.file_menu.add_command(label="Save", command=self.file_save, accelerator="Ctrl+S")
        self.menubar.add_cascade(label="File", menu=self.file_menu)
        self.file_menu.add_separator()
        
        self.filemenu = tk.Menu(self.menubar, tearoff=0, bg="lightgrey", fg="black")
        #app = App()
        self.filemenu.add_command(label="Excel Display", command=self.html_app, accelerator="Ctrl+E")
        self.menubar.add_cascade(label="Excel-html", menu=self.filemenu)
        
        
        self.editmenu = tk.Menu(self.menubar, tearoff=0, bg="lightgrey", fg="black")
        self.editmenu.add_command(label="Undo", command=self.filemenu)

        self.editmenu.add_separator()

        self.editmenu.add_command(label="Cut", command=self.filemenu)
        self.editmenu.add_command(label="Copy", command=self.filemenu)
        self.editmenu.add_command(label="Paste", command=self.filemenu)
        self.editmenu.add_command(label="Delete", command=self.filemenu)
        self.editmenu.add_command(label="Select All", command=self.filemenu)
        self.menubar.add_cascade(label="Edit", menu=self.editmenu)
       
        self.helpmenu = tk.Menu(self.menubar, tearoff=0, bg="lightgrey", fg="black")
        self.helpmenu.add_command(label="Help Index", command=self.filemenu)
        self.helpmenu.add_command(label="About...", command=self.filemenu)
        self.menubar.add_cascade(label="Help", menu=self.helpmenu)
        
        self.language = tk.Menu(self.menubar, tearoff=0, bg="lightgrey", fg="black")
        self.language.add_command(label="Language", command=self.Translate_Book)
        self.menubar.add_cascade(label="language converter", menu=self.language)
        
        self.chart = tk.Menu(self.menubar, tearoff=0, bg="lightgrey", fg="black")
        self.chart.add_command(label="Chart", command=self.vp_chart)
        self.menubar.add_cascade(label="All chart", menu=self.chart)
        
        self.forms = tk.Menu(self.menubar, tearoff=0, bg="lightgrey", fg="black")
        self.forms.add_command(label="Customer Details", command=self.excel_forms)
        self.menubar.add_cascade(label="Forms", menu=self.forms)
        
        self.configure(menu=self.menubar)

        self.main_text = tk.Text(self, bg="white", fg="black", font=("Ubuntu Mono", self.FONT_SIZE))

        self.main_text.pack(expand=1, fill=tk.BOTH)

        self.main_text.bind("<space>", self.destroy_autocomplete_menu)
        self.main_text.bind("<KeyRelease>", self.display_autocomplete_menu)
        self.main_text.bind("<Tab>", self.insert_spaces)

        self.bind("<Control-s>", self.file_save)
        self.bind("<Control-o>", self.file_open)
        self.bind("<Control-n>", self.file_new)
        
    def html_app(self, event=None):     
        self.app = App()
        
    def Translate_Book(self, event=None):
        self.Translate = TranslateBook()
        
    def vp_chart(self, event=None):
        self.chat = vp_start_gui()
        
    def excel_forms(self, event=None):
        self.forms = os.system('python /home/subir/Downloads/form.py')
        
    def file_new(self, event=None):
        file_name = filedialog.asksaveasfilename()
        if file_name:
            self.open_file = file_name
            self.main_text.delete(1.0, tk.END)
            self.title(" - ".join([self.WINDOW_TITLE, self.open_file]))

    def file_open(self, event=None):
        file_to_open = filedialog.askopenfilename()

        if file_to_open:
            self.open_file = file_to_open
            self.main_text.delete(1.0, tk.END)

            with open(file_to_open, "r") as file_contents:
                file_lines = file_contents.readlines()
                if len(file_lines) > 0:
                    for index, line in enumerate(file_lines):
                        index = float(index) + 1.0
                        self.main_text.insert(index, line)

        self.title(" - ".join([self.WINDOW_TITLE, self.open_file]))

    def file_save(self, event=None):
        if not self.open_file:
            new_file_name = filedialog.asksaveasfilename()
            if new_file_name:
                self.open_file = new_file_name

        if self.open_file:
            new_contents = self.main_text.get(1.0, tk.END)
            with open(self.open_file, "w") as open_file:
                open_file.write(new_contents)

    def insert_spaces(self, event=None):
        self.main_text.insert(tk.INSERT, "    ")

        return "break"

    def get_menu_coordinates(self):
        bbox = self.main_text.bbox(tk.INSERT)
        menu_x = bbox[0] + self.winfo_x() + self.main_text.winfo_x()
        menu_y = bbox[1] + self.winfo_y() + self.main_text.winfo_y() + self.FONT_SIZE + 2

        return (menu_x, menu_y)

    def display_autocomplete_menu(self, event=None):
        current_index = self.main_text.index(tk.INSERT)
        start = self.adjust_floating_index(current_index)

        try:
            currently_typed_word = self.main_text.get(start + " wordstart", tk.INSERT)
        except tk.TclError:
            currently_typed_word = ""

        currently_typed_word = str(currently_typed_word).strip()

        if currently_typed_word:
            self.destroy_autocomplete_menu()

            suggestions = []
            for word in self.AUTOCOMPLETE_WORDS:
                if word.startswith(currently_typed_word) and not currently_typed_word == word:
                    suggestions.append(word)

            if len(suggestions) > 0:
                x, y = self.get_menu_coordinates()
                self.complete_menu = tk.Menu(self, tearoff=0, bg="lightgrey", fg="black")

                for word in suggestions:
                    insert_word_callback = partial(self.insert_word, word=word, part=currently_typed_word, index=current_index)
                    self.complete_menu.add_command(label=word, command=insert_word_callback)

                self.complete_menu.post(x, y)
                self.main_text.bind("<Down>", self.focus_menu_item)

    def destroy_autocomplete_menu(self, event=None):
        try:
            self.complete_menu.destroy()
            self.main_text.unbind("<Down>")
            self.main_text.focus_force()
        except AttributeError:
            pass

    def insert_word(self, word, part, index):
        amount_typed = len(part)
        remaining_word = word[amount_typed:]
        remaining_word_offset = " +" + str(len(remaining_word)) + "c"
        self.main_text.insert(index, remaining_word)
        self.main_text.mark_set(tk.INSERT, index + remaining_word_offset)
        self.destroy_autocomplete_menu()
        self.main_text.focus_force()

    def adjust_floating_index(self, number):
        indices = number.split(".")
        x_index = indices[0]
        y_index = indices[1]
        y_as_number = int(y_index)
        y_previous = y_as_number - 1
        
        return ".".join([x_index, str(y_previous)])

    def focus_menu_item(self, event=None):
        try:
            self.complete_menu.focus_force()
            self.complete_menu.entryconfig(0, state="active")
        except tk.TclError:
            pass
   
  
    # start the GUI 
if __name__ == "__main__":
        editor = Editor()
        editor.mainloop()
        #label.pack()
        root.mainloop()
    #top.mainloop()


NameError: name 'root' is not defined

In [ ]:
################# Chart windows
import sys
from tkinter.filedialog import askopenfilename

try:
    from Tkinter import *
except ImportError:
    from tkinter import *

try:
    import ttk
    py3 = 0
except ImportError:
    import tkinter.ttk as ttk
    py3 = 1

def vp_start_gui():
    '''Starting point when module is the main routine.'''
    global val, w, root
    root = Tk()
    top = New_Toplevel_1 (root)
    root.mainloop()

w = None
def create_New_Toplevel_1(root, *args, **kwargs):
    '''Starting point when module is imported by another program.'''
    global w, w_win, rt
    rt = root
    w = Toplevel (root)
    top = New_Toplevel_1 (w)
    return (w, top)

def destroy_New_Toplevel_1():
    global w
    w.destroy()
    w = None

class New_Toplevel_1:
    def __init__(self, top=None):
        '''This class configures and populates the toplevel window.
           top is the toplevel containing window.'''
        self._bgcolor = '#d9d9d9'  # X11 color: 'gray85'
        self._fgcolor = '#000000'  # X11 color: 'black'
        self._compcolor = '#d9d9d9' # X11 color: 'gray85'
        self._ana1color = '#d9d9d9' # X11 color: 'gray85' 
        self._ana2color = '#d9d9d9' # X11 color: 'gray85' 

        top.geometry("757x1037+832+67")
        top.title("New Toplevel 1")
        top.configure(background="#d9d9d9")
        top.configure(highlightbackground="#d9d9d9")
        top.configure(highlightcolor="black")


        self.Canvas1 = Canvas(top)
        self.Canvas1.place(relx=0.04, rely=0.58, relheight=0.4, relwidth=0.92)
        self.Canvas1.configure(background="white")
        self.Canvas1.configure(borderwidth="2")
        self.Canvas1.configure(highlightbackground="#d9d9d9")
        self.Canvas1.configure(highlightcolor="black")
        self.Canvas1.configure(insertbackground="black")
        self.Canvas1.configure(relief=RIDGE)
        self.Canvas1.configure(selectbackground="#c4c4c4")
        self.Canvas1.configure(selectforeground="black")
        self.Canvas1.configure(width=695)

        self.Button2 = Button(top)
        self.Button2.place(relx=0.75, rely=0.52, height=42, width=138)
        self.Button2.configure(activebackground="#d9d9d9")
        self.Button2.configure(activeforeground="#000000")
        self.Button2.configure(background="#d9d9d9")
        self.Button2.configure(disabledforeground="#a3a3a3")
        self.Button2.configure(foreground="#000000")
        self.Button2.configure(highlightbackground="#d9d9d9")
        self.Button2.configure(highlightcolor="black")
        self.Button2.configure(pady="0")
        self.Button2.configure(text='''Generate Graph''')

        self.Labelframe1 = LabelFrame(top)
        self.Labelframe1.place(relx=0.05, rely=0.39, relheight=0.18, relwidth=0.44)
        self.Labelframe1.configure(relief=GROOVE)
        self.Labelframe1.configure(foreground="black")
        self.Labelframe1.configure(text='''Type of Graph''')
        self.Labelframe1.configure(background="#d9d9d9")
        self.Labelframe1.configure(highlightbackground="#d9d9d9")
        self.Labelframe1.configure(highlightcolor="black")
        self.Labelframe1.configure(width=330)

        self.Radiobutton1 = Radiobutton(self.Labelframe1)
        self.Radiobutton1.place(relx=0.06, rely=0.22, relheight=0.2, relwidth=0.31)
        self.Radiobutton1.configure(activebackground="#d9d9d9")
        self.Radiobutton1.configure(activeforeground="#000000")
        self.Radiobutton1.configure(background="#d9d9d9")
        self.Radiobutton1.configure(disabledforeground="#a3a3a3")
        self.Radiobutton1.configure(foreground="#000000")
        self.Radiobutton1.configure(highlightbackground="#d9d9d9")
        self.Radiobutton1.configure(highlightcolor="black")
        self.Radiobutton1.configure(justify=LEFT)
        self.Radiobutton1.configure(text='''Bar Chart''')

        self.Radiobutton2 = Radiobutton(self.Labelframe1)
        self.Radiobutton2.place(relx=0.06, rely=0.38, relheight=0.2, relwidth=0.35)
        self.Radiobutton2.configure(activebackground="#d9d9d9")
        self.Radiobutton2.configure(activeforeground="#000000")
        self.Radiobutton2.configure(background="#d9d9d9")
        self.Radiobutton2.configure(disabledforeground="#a3a3a3")
        self.Radiobutton2.configure(foreground="#000000")
        self.Radiobutton2.configure(highlightbackground="#d9d9d9")
        self.Radiobutton2.configure(highlightcolor="black")
        self.Radiobutton2.configure(justify=LEFT)
        self.Radiobutton2.configure(text='''Histogram''')

        self.Radiobutton3 = Radiobutton(self.Labelframe1)
        self.Radiobutton3.place(relx=0.06, rely=0.54, relheight=0.2, relwidth=0.37)
        self.Radiobutton3.configure(activebackground="#d9d9d9")
        self.Radiobutton3.configure(activeforeground="#000000")
        self.Radiobutton3.configure(background="#d9d9d9")
        self.Radiobutton3.configure(disabledforeground="#a3a3a3")
        self.Radiobutton3.configure(foreground="#000000")
        self.Radiobutton3.configure(highlightbackground="#d9d9d9")
        self.Radiobutton3.configure(highlightcolor="black")
        self.Radiobutton3.configure(justify=LEFT)
        self.Radiobutton3.configure(text='''Scatter Plot''')

        self.Button3 = Button(top)
        self.Button3.place(relx=0.28, rely=0.05, height=52, width=122)
        self.Button3.configure(activebackground="#d9d9d9")
        self.Button3.configure(activeforeground="#000000")
        self.Button3.configure(background="#d9d9d9")
        self.Button3.configure(disabledforeground="#a3a3a3")
        self.Button3.configure(foreground="#000000")
        self.Button3.configure(highlightbackground="#d9d9d9")
        self.Button3.configure(highlightcolor="black")
        self.Button3.configure(pady="0")
        self.Button3.configure(text='''Browse''')
        self.Button3.configure(width=122)
        self.Button3.configure(command=askopenfilename)

        self.Label5 = Label(top)
        self.Label5.place(relx=0.03, rely=0.06, height=31, width=147)
        self.Label5.configure(activebackground="#f9f9f9")
        self.Label5.configure(activeforeground="black")
        self.Label5.configure(background="#d9d9d9")
        self.Label5.configure(disabledforeground="#a3a3a3")
        self.Label5.configure(foreground="#000000")
        self.Label5.configure(highlightbackground="#d9d9d9")
        self.Label5.configure(highlightcolor="black")
        self.Label5.configure(text='''Upload File:''')
        self.Label5.configure(width=147)

        self.Label3 = Label(top)
        self.Label3.place(relx=0.05, rely=0.13, height=31, width=111)
        self.Label3.configure(background="#d9d9d9")
        self.Label3.configure(disabledforeground="#a3a3a3")
        self.Label3.configure(foreground="#000000")
        self.Label3.configure(text='''Data Frame :''')

        self.Text1 = Text(top)
        self.Text1.place(relx=0.05, rely=0.16, relheight=0.21, relwidth=0.9)
        self.Text1.configure(background="white")
        self.Text1.configure(font="TkTextFont")
        self.Text1.configure(foreground="black")
        self.Text1.configure(highlightbackground="#d9d9d9")
        self.Text1.configure(highlightcolor="black")
        self.Text1.configure(insertbackground="black")
        self.Text1.configure(selectbackground="#c4c4c4")
        self.Text1.configure(selectforeground="black")
        self.Text1.configure(width=684)
        self.Text1.configure(wrap=WORD)

        self.Labelframe2 = LabelFrame(top)
        self.Labelframe2.place(relx=0.5, rely=0.39, relheight=0.12, relwidth=0.45)
        self.Labelframe2.configure(relief=GROOVE)
        self.Labelframe2.configure(foreground="black")
        self.Labelframe2.configure(text='''Labelframe''')
        self.Labelframe2.configure(background="#d9d9d9")
        self.Labelframe2.configure(width=340)

        self.Label1 = Label(self.Labelframe2)
        self.Label1.place(relx=0.03, rely=0.24, height=31, width=67)
        self.Label1.configure(background="#d9d9d9")
        self.Label1.configure(disabledforeground="#a3a3a3")
        self.Label1.configure(foreground="#000000")
        self.Label1.configure(text='''X-axis :''')

        self.Label2 = Label(self.Labelframe2)
        self.Label2.place(relx=0.03, rely=0.56, height=31, width=66)
        self.Label2.configure(background="#d9d9d9")
        self.Label2.configure(disabledforeground="#a3a3a3")
        self.Label2.configure(foreground="#000000")
        self.Label2.configure(text='''Y-axis :''')

        self.Entry1 = Entry(self.Labelframe2)
        self.Entry1.place(relx=0.24, rely=0.24, relheight=0.29, relwidth=0.72)
        self.Entry1.configure(background="white")
        self.Entry1.configure(disabledforeground="#a3a3a3")
        self.Entry1.configure(font="TkFixedFont")
        self.Entry1.configure(foreground="#000000")
        self.Entry1.configure(insertbackground="black")
        self.Entry1.configure(width=244)

        self.Entry2 = Entry(self.Labelframe2)
        self.Entry2.place(relx=0.24, rely=0.56, relheight=0.29, relwidth=0.72)
        self.Entry2.configure(background="white")
        self.Entry2.configure(disabledforeground="#a3a3a3")
        self.Entry2.configure(font="TkFixedFont")
        self.Entry2.configure(foreground="#000000")
        self.Entry2.configure(insertbackground="black")
        self.Entry2.configure(width=244)

if __name__ == '__main__':
    vp_start_gui()

In [ ]:
############## convert excel
#!/usr/bin/env python3

from xlrd import open_workbook
from subprocess import call
from os import remove
from os import getlogin
from os import name as os_name
import tkinter
from tkinter import ttk
from tkinter import messagebox
from tkinter import filedialog
from time import sleep


class App(object):

    def __init__(self):
        self.root = tkinter.Tk()
        self.root.minsize(250,1)
        self.root.title('Excel HTML Table View')

        frm = ttk.Frame(self.root)
        frm.grid(column=0, row=0, sticky=(tkinter.N, tkinter.W, tkinter.E, tkinter.S))
        frm.columnconfigure(0, weight=1)
        frm.rowconfigure(0, weight=1)
        
        self.filename_var = tkinter.StringVar()
        self.filename_var.set("<select an input file>")

        open_button = ttk.Button(frm, text="Input File", command=self.select_file)
        open_button.grid(column=1, row=1, sticky=tkinter.W)
        open_button['command'] = self.select_file

        display_box = ttk.Label(frm, textvariable=self.filename_var, width=30)
        display_box.grid(column=2, row=1, sticky=((tkinter.W, tkinter.E)))

        exit_button = ttk.Button(frm, text='Close')
        exit_button.grid(column=1, row=2, sticky=tkinter.W)
        exit_button['command'] = self.exit

    def exit(*args):
        quit()
        
    def select_file(self):
        if os_name == 'nt':
            filename = filedialog.askopenfilename(filetypes=[('Excel Spreadsheet','*.xlsx'),('Excel Spreadsheet','*.xls'), ('All files','*.*')], initialdir='C:/Users/{}/documents'.format(getlogin()))
        else:
            filename = filedialog.askopenfilename(filetypes=[('Excel Spreadsheet','*.xlsx'),('Excel Spreadsheet','*.xls'), ('All files','*.*')])
        self.filename_var.set(filename)
        self.make_html(filename)

    def make_html(self, excel_file):
        html_file = str(excel_file.split('.')[0]+'.html')
        book = open_workbook(excel_file)
        f = open(html_file, 'w')
        f.write('<!DOCTYPE html>\n\
        <head>\n\
        <style>\n\
        body {\n\
            font-family: Sans-Serif;\n\
        }\n\
        \n\
        table.container {\n\
            border-collapse: collapse;\n\
            border-spacing: 0;\n\
            width: 100%;\n\
            border: 3px solid #9C9C9C;\n\
        }\n\
        \n\
        td.bordered {\n\
            text-align: center;\n\
            vertical-align: middle;\n\
            padding: 0px;\n\
            border: 2px solid #9C9C9C;\n\
        }\n\
        \n\
        td.empty {\n\
            text-align: center;\n\
            vertical-align: middle;\n\
            padding: 0px;\n\
            border: 1px solid #BBBBBB;\n\
        }\n\
        \n\
        </style>')
        f.write('<title>{}</title>'.format(excel_file))
        f.write('</head>\n\
        <body>\n')
        
        for sheet_n in range(book.nsheets):
            sheet = book.sheet_by_index(sheet_n)
            if sheet.ncols != 0:
                f.write('<table class="container">\n')
                f.write('<tr>\n<th>{}</th>\n</tr>\n'.format(sheet.name))
                for row in range(sheet.nrows):
                    f.write('<tr>\n')
                    for column in range(sheet.ncols):
                        try:
                            if (sheet.cell(row, column).value) == '' :
                                f.write('<td class="empty">')
                            else:
                                f.write('<td class="bordered">')
                            f.write(str(sheet.cell(row, column).value))
                            f.write('</td>\n')
                        except:
                            pass
                    f.write('</tr>\n')
                f.write('</table>\n<br />\n<br />\n')
        f.write('</body>\n')
        f.write('</html>\n')
        f.close()
        if os_name == 'nt':
            call(html_file, shell='True')
            remove(html_file)
        else:
            call('nohup xdg-open {}'.format(html_file), shell='True')
            sleep(1)
            remove(html_file)


app = App()
app.root.mainloop()


In [ ]:
import sys
import mysql.connector
from xlsxwriter.workbook import Workbook

user = '' # your username
passwd = '' # your password
host = '' # your host
db = '' # database where your table is stored
table = '' # table you want to save

dbconnect = mysql.connector.connect(option_files='/home/subir/anaconda3/notebook/my.conf',
                              raise_on_warnings=True,connect_timeout=2000,buffered=True,auth_plugin='mysql_native_password')
cursor = dbconnect.cursor()

query = "SELECT * FROM course_enrollment"
cursor.execute(query)

workbook = Workbook('/home/subir/anaconda3/outfile.xlsx')
sheet = workbook.add_worksheet()
for r, row in enumerate(cursor.fetchall()):
    for c, col in enumerate(row):
        sheet.write(r, c, col)
print(row)

In [ ]:
############  This is dummy File manupulation
from tkinter import *

def donothing():
    filewin = Toplevel(root)
    button = Button(filewin, text="Do nothing button")
    button.pack()  
root = Tk()
menubar = Menu(root)
filemenu = Menu(menubar, tearoff=0)
filemenu.add_command(label="New", command=donothing)
filemenu.add_command(label="Open", command=donothing)
filemenu.add_command(label="Save", command=donothing)
filemenu.add_command(label="Save as...", command=donothing)
filemenu.add_command(label="Close", command=donothing)

filemenu.add_separator()

filemenu.add_command(label="Exit", command=root.quit)
menubar.add_cascade(label="File", menu=filemenu)
editmenu = Menu(menubar, tearoff=0)
editmenu.add_command(label="Undo", command=donothing)

editmenu.add_separator()

editmenu.add_command(label="Cut", command=donothing)
editmenu.add_command(label="Copy", command=donothing)
editmenu.add_command(label="Paste", command=donothing)
editmenu.add_command(label="Delete", command=donothing)
editmenu.add_command(label="Select All", command=donothing)

menubar.add_cascade(label="Edit", menu=editmenu)
helpmenu = Menu(menubar, tearoff=0)
helpmenu.add_command(label="Help Index", command=donothing)
helpmenu.add_command(label="About...", command=donothing)
menubar.add_cascade(label="Help", menu=helpmenu)

root.config(menu=menubar)
root.mainloop()

In [ ]:
#########  Translate form This is done and working ###############
import tkinter as tk
from tkinter import messagebox as msg
from tkinter.ttk import Notebook

import requests

class TranslateBook(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Translation Book v1")
        self.geometry("500x300")

        self.notebook = Notebook(self)

        english_tab = tk.Frame(self.notebook)
        italian_tab = tk.Frame(self.notebook)

        self.translate_button = tk.Button(english_tab, text="Translate", command=self.translate)
        self.translate_button.pack(side=tk.BOTTOM, fill=tk.X)

        self.english_entry = tk.Text(english_tab, bg="white", fg="black")
        self.english_entry.pack(side=tk.TOP, expand=1)

        self.italian_copy_button = tk.Button(italian_tab, text="Copy to Clipboard", command=self.copy_to_clipboard)
        self.italian_copy_button.pack(side=tk.BOTTOM, fill=tk.X)

        self.italian_translation = tk.StringVar(italian_tab)
        self.italian_translation.set("")

        self.italian_label = tk.Label(italian_tab, textvar=self.italian_translation, bg="lightgrey", fg="black")
        self.italian_label.pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        self.notebook.add(english_tab, text="English")
        self.notebook.add(italian_tab, text="Italian")

        self.notebook.pack(fill=tk.BOTH, expand=1)

    def translate(self, target_language="it", text=None):
        if not text:
            text = self.english_entry.get(1.0, tk.END)

        url = "https://translate.googleapis.com/translate_a/single?client=gtx&sl={}&tl={}&dt=t&q={}".format("en", target_language, text)

        try:
            r = requests.get(url)
            r.raise_for_status()
            translation = r.json()[0][0][0]
            self.italian_translation.set(translation)
            msg.showinfo("Translation Successful", "Text successfully translated")
        except Exception as e:
            msg.showerror("Translation Failed", str(e))

    def copy_to_clipboard(self, text=None):
        if not text:
            text = self.italian_translation.get()

        self.clipboard_clear()
        self.clipboard_append(text)
        msg.showinfo("Copied Successfully", "Text copied to clipboard")


if __name__ == "__main__":
    translatebook = TranslateBook()
    translatebook.mainloop()